In [3]:
import requests
import bs4
from bs4 import BeautifulSoup

In [2]:
def get_symptoms(URL):
    r = requests.get(URL)

    soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib
#     print(soup.prettify())

    name = soup.find('h1').get_text()
    
    content = soup.find('div', class_='content')
#     content = content.find('h2').previous_element.previous_element.previous_element
    
    h2_list = content.find_all('h2')
    
    symp_index = -1
    for i in range(len(h2_list)):
        if h2_list[i].get_text() == 'Symptoms':
            symp_index = i
            break

    if symp_index == -1:
        return {}
    else:
        visually_hidden = content.find_all("span", {"class": "myc-visuallyhidden"})
        weird_error = content.find_all("p", {"class": "myc-error-header"})
        form_error = ["myc-subscription-form", "acces-list-container", "requestappt"]
        
        h2_index = symp_index
        element = h2_list[h2_index]

        parser_dict = {}
        parser_dict['illness'] = name
        
        images = content.find_all("img")
        
        if len(images) > 0:
            parser_dict['img'] = "https://www.mayoclinic.org" + images[0]['src']
        
        parser_dict['url'] = URL
        
        section_str = ""
        form_str = ""
        
        form = False

        while h2_index == symp_index:
            element = element.next_element
            if element == h2_list[h2_index + 1]:
                parser_dict[h2_list[h2_index].get_text().lower()] = section_str.replace("  ", " ").replace(" , ", ", ").replace(" . ", ". ")
                h2_index += 1
                section_str = ''
                
                form = False

        #         if element.get_text() == '':
        #             empty_header = True

            elif isinstance(element, bs4.element.NavigableString):
                if not isinstance(element, bs4.element.Comment):
                    if (element != h2_list[h2_index].get_text()):
                        if not element.isspace():
                            if form:
                                form_str += element
                            else:
                                section_str += element.strip() + ' '

            elif element in visually_hidden:
                element = element.next_element
#                 print('Got one!')

            elif element in weird_error:
                element = element.next_element
#                 print('Oh!')
            
            elif isinstance(element, bs4.element.Tag):
                if element.name == "style":
                    element = element.next_element
                elif element.name == "div" and element.has_attr("class"):
                    if any(item in form_error for item in element["class"]):
                        form = True
            
                
        return parser_dict

In [7]:
# URLS = []
# with open('top_10.txt', mode='r') as top10:
#     for illness in top10:
#         URLS.append(illness.strip())

In [8]:
# top10_symptoms = []
# for URL in URLS:
#     top10_symptoms.append(get_symptoms(URL))

In [13]:
# top10_symptoms

In [10]:
# import json

# with open('updated_illness.json', 'w') as data:
#     json.dump(top10_symptoms, data)

In [12]:
test = get_symptoms("https://www.mayoclinic.org/diseases-conditions/tricuspid-valve-disease/symptoms-causes/syc-20350609")

AttributeError: 'NoneType' object has no attribute 'find_all'

In [10]:
test

{'illness': 'Mosquito bites',
 'symptoms': 'Mosquito bite signs include: A puffy, white and reddish bump that appears a few minutes after the bite A hard, itchy, reddish-brown bump, or multiple bumps, appearing a day or so after the bite or bites Small blisters instead of hard bumps Dark spots that look like bruises More-severe reactions may be experienced by children, adults not previously exposed to the type of mosquito that bit them, and people with immune system disorders. In these people, mosquito bites sometimes trigger: A large area of swelling and redness Low-grade fever Hives Swollen lymph nodes Children are more likely to develop a severe reaction than are adults, because many adults have had mosquito bites throughout their lives and become desensitized. When to see a doctor If mosquito bites seem to be associated with more-serious warning signs — such as fever, headache, body aches and signs of infection — contact your doctor. Request an Appointment at Mayo Clinic '}